In [1]:
import pandas as pd
import matplotlib as plt
import scipy.stats as stats
import numpy as np

In [2]:
path = 'https://stepik.org/media/attachments/lesson/406362/churn.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [5]:
df.shape

(50000, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_dist                50000 non-null  float64
 1   avg_rating_by_driver    49799 non-null  float64
 2   avg_rating_of_driver    41878 non-null  float64
 3   avg_surge               50000 non-null  float64
 4   city                    50000 non-null  object 
 5   last_trip_date          50000 non-null  object 
 6   phone                   49604 non-null  object 
 7   signup_date             50000 non-null  object 
 8   surge_pct               50000 non-null  float64
 9   trips_in_first_30_days  50000 non-null  int64  
 10  luxury_car_user         50000 non-null  bool   
 11  weekday_pct             50000 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [7]:
df.last_trip_date = pd.to_datetime(df.last_trip_date)
df.signup_date = pd.to_datetime(df.signup_date)

In [8]:
df.last_trip_date.max()

Timestamp('2014-07-01 00:00:00')

In [9]:
df['days_since_last_trip'] = df.last_trip_date.max() - df.last_trip_date

In [10]:
df['days_since_last_trip']

0        14 days
1        57 days
2       175 days
3         2 days
4       108 days
          ...   
49995    26 days
49996   157 days
49997    40 days
49998   167 days
49999    72 days
Name: days_since_last_trip, Length: 50000, dtype: timedelta64[ns]

In [11]:
df['days_since_last_trip'] = df['days_since_last_trip'].dt.days

In [12]:
df['days_since_last_trip']

0         14
1         57
2        175
3          2
4        108
        ... 
49995     26
49996    157
49997     40
49998    167
49999     72
Name: days_since_last_trip, Length: 50000, dtype: int64

In [13]:
df['churn'] = df.days_since_last_trip.apply(lambda x: 'churn' if x > 30 else 'not_churn')

In [14]:
df[['city', 'churn']]

,city,churn
0,King's Landing,not_churn
1,Astapor,churn
2,Astapor,churn
3,King's Landing,not_churn
4,Winterfell,churn
...,...,...
49995,King's Landing,not_churn
49996,Astapor,churn
49997,Winterfell,churn
49998,Astapor,churn


In [27]:
x1 = pd.crosstab(index=df.city, columns=df.churn); x1

churn,churn,not_churn
city,,
Astapor,12306,4228
King's Landing,3767,6363
Winterfell,15123,8213


In [28]:
stats.chi2_contingency(x1)[1] #p-value

0.0

In [25]:
for i in range(10):
    print(stats.shapiro(df.trips_in_first_30_days.sample(1000))[1]) #p-values

0.0
1.14906474074635e-43
1.401298464324817e-45
4.764414778704378e-44
1.401298464324817e-45
7.228023595849196e-40
2.5643761897144152e-43
1.8216880036222622e-44
1.6815581571897805e-44
1.0089348943138683e-43


In [60]:
x2 = pd.crosstab(index=df.trips_in_first_30_days, columns=df.city); x2.head()

city,Astapor,King's Landing,Winterfell
trips_in_first_30_days,,,
0,5333,3532,6525
1,5143,2270,6695
2,2445,1466,3491
3,1153,764,1871
4,738,551,1273


In [61]:
x2.columns

Index(['Astapor', 'King's Landing', 'Winterfell'], dtype='object', name='city')

In [67]:
stats.kruskal(x2[x2.columns[0]],
              x2[x2.columns[1]],
              x2[x2.columns[2]])

KruskalResult(statistic=1.7156165755559356, pvalue=0.4240905489195994)

In [54]:
x3 = df[['trips_in_first_30_days', 'churn']]; x3.head()

,trips_in_first_30_days,churn
0,4,not_churn
1,0,churn
2,3,churn
3,9,not_churn
4,14,churn


In [55]:
x3 = pd.crosstab(x3.trips_in_first_30_days, x3.churn); x3.head()

churn,churn,not_churn
trips_in_first_30_days,,
0,9810,5580
1,10983,3125
2,4659,2743
3,2095,1693
4,1249,1313


In [58]:
x3 = x3.reset_index()

In [59]:
stats.mannwhitneyu(x3.churn, x3.not_churn)

MannwhitneyuResult(statistic=1358.0, pvalue=0.019507017720223237)